### 安装依赖包 

```sh
uv add alibabacloud_alidns20150109 alibabacloud_credentials alibabacloud_tea_openapi alibabacloud_tea_util alibabacloud_tea_console
```

初始化配置对象 ```alibabacloud_tea_openapi.Config``` 。
```Config``` 对象存放存放 ```access_key_id``` 、```access_key_secret``` 和 ```endpoint``` 等配置，Endpoint 请参考 https://api.aliyun.com/product/Alidns

In [ ]:
import os
import dotenv
from alibabacloud_tea_openapi import models as open_api_models

dotenv.load_dotenv()

access_key_id = os.getenv('ACCESS_KEY_ID')
access_key_secret = os.getenv('ACCESS_KEY_SECRET')

if access_key_id is None or access_key_secret is None:
    raise ValueError("ACCESS_KEY_ID and ACCESS_KEY_SECRET must be set in environment variables.")


config = open_api_models.Config(
    access_key_id=access_key_id,
    access_key_secret=access_key_secret,
    endpoint='dns.aliyuncs.com'
)

实例化一个客户端，从 ```alibabacloud_alidns20150109.Client``` 类生成对象 ```client``` 。 
后续 ```request```、```response``` 从 ```alibabacloud_alidns20150109.models``` 中获得。

In [ ]:
from alibabacloud_alidns20150109.client import Client

client = Client(config)

创建对应 API 的 ```Request``` 。 方法的命名规则为 API 名称加上 ```Request``` 。

In [ ]:
from alibabacloud_alidns20150109 import models

request = models.DescribeDomainRecordsRequest()

设置请求类 ```request``` 的参数。 通过设置 ```request``` 类的属性设置参数，即 API 中必须要提供的信息。例如：

In [ ]:
request.lang = 'zh'
request.domain_name = '3ncounter.cn'

通过 client 对象获得对应 ```request``` 响应 response 。

In [ ]:
response = client.describe_domain_records(request)
print(response.body)

In [ ]:
def getDomainRecords():
    # 解析记录列表
    request = models.DescribeDomainRecordsRequest()
    request.lang = 'zh'
    request.domain_name = '3ncounter.cn'
    response = client.describe_domain_records(request)
    return response

def deleteDomainRecords(record_id):
    # 删除解析记录
    request = models.DeleteDomainRecordRequest()
    request.record_id = record_id
    response = client.delete_domain_record(request)
    return response

def updateDomainRecord(record_id, rr, value, ttl):
    # 更新解析记录
    request = models.UpdateDomainRecordRequest()
    request.record_id = record_id
    request.rr = rr
    request.type = 'AAAA'
    request.value = value
    request.ttl = ttl
    response = client.update_domain_record(request)
    return response

In [ ]:
# import requests

# res = requests.get('http://ip-api.com/json')
# print(res.json())

# 获取域名解析记录列表
res = getDomainRecords()

aaaaRecord = None

for item in res.body.domain_records.record:
    if item.type == 'AAAA':
        aaaaRecord = item
        break

if aaaaRecord is not None:
    print(aaaaRecord)

{'status': 'success', 'country': 'China', 'countryCode': 'CN', 'region': 'CQ', 'regionName': 'Chongqing', 'city': 'Chongqing', 'zip': '', 'lat': 29.5657, 'lon': 106.551, 'timezone': 'Asia/Shanghai', 'isp': 'China Unicom Chongqing Province Network', 'org': '', 'as': 'AS4837 CHINA UNICOM China169 Backbone', 'query': '27.10.94.48'}
{'CreateTimestamp': 1750923857189, 'DomainName': '3ncounter.cn', 'Line': 'default', 'Locked': False, 'RR': 'ddns', 'RecordId': '1938141249295366144', 'Status': 'ENABLE', 'TTL': 600, 'Type': 'AAAA', 'UpdateTimestamp': 1750923857412, 'Value': '240e:331:208e:b900:820f:42a2:d3f7:a30b', 'Weight': 1}


In [ ]:
# 删除指定的解析记录
res = deleteDomainRecords('895164709898411008')

In [48]:
import socket

res = socket.getaddrinfo('localhost', 80)
res

[(<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_DGRAM: 2>,
  17,
  '',
  ('127.0.0.1', 80)),
 (<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('127.0.0.1', 80)),
 (<AddressFamily.AF_INET6: 30>,
  <SocketKind.SOCK_DGRAM: 2>,
  17,
  '',
  ('::1', 80, 0, 0)),
 (<AddressFamily.AF_INET6: 30>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('::1', 80, 0, 0))]

使用 try...except... 处理报错。

In [ ]:

# from Tea.exceptions import UnretryableException

# try:
#     response = client.add_custom_line(request)
#     print(response)
# except UnretryableException as e:
#     print(e)
# except Exception as e:
#     print(e)

In [90]:
import requests
import json

ipv6 = 'http://ipv6.lookup.test-ipv6.com/ip/?callback=_jqjsp&asn=1&testdomain=test-ipv6.fratec.net&testname=test_asn6'
ipv4 = 'http://ipv4.lookup.test-ipv6.com/ip/?callback=_jqjsp&asn=1&testdomain=test-ipv6.fratec.net&testname=test_asn4'


ipv4res = requests.get(ipv4)
ipv6res = requests.get(ipv6)

print(json.loads(ipv4res.text.replace('_jqjsp(', '').replace(')', ''))['ip'])
print(json.loads(ipv6res.text.replace('_jqjsp(', '').replace(')', ''))['ip'])


46.232.121.49
2407:cdc0:b010::49
